In [ ]:
import os
import cProfile
import weave
import random
import time
from PIL import Image

def create_data():
    ims = []
    base_im = Image.linear_gradient("L")
    random.seed(0)
    for rotate in range(5):
        for shear in range(5):
            for i in range(2000):
                ims.append(
                    {
                        "rotate": rotate,
                        "shear": shear,
                        "y": random.choice(["a", "b", "c"]),
                        "x": random.randrange(10),
#                         "image": base_im.rotate(rotate * 4).transform(
#                             (256, 256),
#                             Image.AFFINE,
#                             (1, shear / 10, 0, 0, 1, 0),
#                             Image.BICUBIC,
#                         ),
                    }
                )
    from weave import storage

    return storage.save(ims)


def run(ref):
#     node = (
#         weave.get(ref)
#         .groupby(lambda row: weave.ops.dict_(rotate=row["rotate"], shear=row["shear"]))
#         .map(lambda row: row.groupby(lambda row: row["y"]))
#         .dropna()
#         .count()
#     )
    node = (
        weave.get(ref)
        .groupby(lambda row: weave.ops.dict_(rotate=row["rotate"], shear=row["shear"]))
        .count()
    )
    result = weave.use(node)
    print('Run result: %s' % result)


os.system("rm -rf /tmp/local-artifacts/*")
ref = create_data()
print("Data ref: ", ref)
start_time = time.time()
cProfile.run('run("%s")' % ref, "profile_stats")
print('Run time: %ss' % (time.time() - start_time))

In [ ]:
import pstats
from pstats import SortKey
p = pstats.Stats('profile_stats')
#p.strip_dirs().sort_stats(SortKey.CUMULATIVE).print_stats()
p.strip_dirs().sort_stats(SortKey.CUMULATIVE).print_callers('type_of')
#p.strip_dirs().sort_stats(SortKey.TIME).print_stats()